# Data Augmentation experiments

In [1]:
!pip install pandas --quiet
!pip install openai --quiet

In [6]:
!pip install pyarrow --quiet

## Imports and data read

In [2]:
import pandas as pd
import numpy as np
import pprint

from tqdm import tqdm
from openai import OpenAI

## Pipeline

https://platform.openai.com/docs/api-reference/authentication

In [3]:
api_key = 'sk-proj-7yyg4KC696T7WK6CmLeWT3BlbkFJfjiC6bs5r8R7yoGLCIUP'

In [4]:
client = OpenAI(api_key=api_key)

In [7]:
df = pd.read_parquet('final_17042025.parquet')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12224 entries, 0 to 12223
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   response_id           12224 non-null  int64 
 1   document_id           12224 non-null  int64 
 2   user_id               12224 non-null  int64 
 3   annotator_sentiment   12224 non-null  object
 4   is_ck_annotation      12224 non-null  int64 
 5   response_timestamp    12224 non-null  object
 6   document_content      12224 non-null  object
 7   annotation_date       12224 non-null  object
 8   username              12224 non-null  object
 9   unique_document_id    12224 non-null  object
 10  language_wc           12224 non-null  object
 11  document_length       12224 non-null  int64 
 12  gpt_labels_v1         12224 non-null  object
 13  language_gpt          12224 non-null  object
 14  language_manual       12224 non-null  object
 15  language              12224 non-null

In [9]:
df.language.unique()

array(['ua', 'ru', 'mixed'], dtype=object)

## Stratification label balancing

The core idea is to reduce the imbalance between classes in the dataset by generating new samples by Chat GPT

In [10]:
df.stratification_label.value_counts()

stratification_label
neutral_ua        3291
negative_ua       2433
positive_ua       1859
negative_ru       1799
neutral_ru        1208
mixed_ua           442
positive_ru        441
negative_mixed     309
neutral_mixed      203
mixed_ru           120
positive_mixed      73
mixed_mixed         46
Name: count, dtype: int64

In [11]:
max_stratification = df.stratification_label.value_counts().max()

In [12]:
max_stratification

np.int64(3291)

In [13]:
classes_to_augment = (df.stratification_label.value_counts() - max_stratification).reset_index()

In [14]:
classes_to_augment['count'] = classes_to_augment['count'].apply(abs) 

In [15]:
classes_to_augment

,stratification_label,count
0,neutral_ua,0
1,negative_ua,858
2,positive_ua,1432
3,negative_ru,1492
4,neutral_ru,2083
5,mixed_ua,2849
6,positive_ru,2850
7,negative_mixed,2982
8,neutral_mixed,3088
9,mixed_ru,3171


In [16]:
for text in df.loc[(df.stratification_label == 'mixed_mixed'), 'document_content']:
    pprint.pprint(text, width=250)
    print('----------------------------')
    print('----------------------------')
    print('----------------------------')

'Навіть в цьому дописи про це ні слова: відкривай та авторизуйся через NFC, а по факту це не до кінця працює.   Хтось побачить цей допис з підписників на канал і поведіться, як я.'
----------------------------
----------------------------
----------------------------
('Дівчата,доброго дня.Чи є тут люди з лівобережжя Херсонщини.?Я вибачаюсь чи можна питання?Хто подавав через Дія на ЄВідновлення,та отримав відмову через те що ,будинок в окупації та не внесено його до державного реєстру!?Ми з Олешок,і куди тільки '
 'не звертаємося нам відмовляють в внесенні будинку в державний реєстр.ЦНАП видав офіційну відмову. Може хтось знак куди було перенесено архів м.Олешки.Раніше в Кропивницьк,зараз кажуть що до Одеси.Але ніхто не знає куди.')
----------------------------
----------------------------
----------------------------
('Ангела Меркель наполягає, що вважає правильним своє рішення не приймати Україну до НАТО у 2008 році, незважаючи на критику з боку Зеленського.  Водночас ексканцлерка під

In [17]:
def analyze_sentiment(comment, system_prompt):
    """
    Sends a request to OpenAI's GPT model to analyze sentiment.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": comment}
        ]
    )   
    return response.choices[0].message.content

In [18]:
prompt_inputs = {
    'negative_ua': [858, 'include only Ukrainian words', 'negative sentiment'],
    'positive_ua': [1432, 'include only Ukrainian words', 'positive sentiment'],
    'negative_ru': [1492, 'include only Russian words', 'negative sentiment'],
    'neutral_ru': [2083, 'include only Russian words', 'neutral sentiment'],
    'mixed_ua': [2849, 'include only Ukrainian words', 'mixed sentiment (express positive and negative emotions in different part of the text output'],
    'positive_ru': [2850, 'include only Russian words', 'positive sentiment'],
    'negative_mixed': [2982, 'include Ukrainian words as well as Russian (e.g.: "Доброго вечора, как делишки?")', 'negative sentiment'],
    'neutral_mixed': [3088, 'include Ukrainian words as well as Russian (e.g.: "Доброго вечора, как делишки?")', 'neutral sentiment'],
    'mixed_ru': [3171, 'include only Russian words', 'mixed sentiment (express positive and negative emotions in different part of the text output'],
    'positive_mixed': [3218, 'include Ukrainian words as well as Russian (e.g.: "Доброго вечора, как делишки?")', 'positive sentiment'],
    'mixed_mixed': [3245, 'include Ukrainian words as well as Russian (e.g.: "Доброго вечора, как делишки?")', 'mixed sentiment (express positive and negative emotions in different part of the text output)'],

}

In [25]:
prompt_outputs = {
    
}

In [ ]:
for strat_label, inputs in tqdm(prompt_inputs.items()):

    prompt_outputs[strat_label] = []
    
    for _ in tqdm(range(inputs[0])):
        language = inputs[1]
        sentiment = inputs[2]
        text = df.loc[(df.stratification_label == strat_label), 'document_content'].sample(1).values[0]
        


        system_prompt_overall = f'''

                You are a sentiment analysis expert. You need to help to create a dataset of texts needed for training an ML model. Your help is to write a text which will be included to the dataset. This is important that the text must {language}. The sentiment of the text should express {sentiment}.
                The example of such a text is provided below.

                Write the text similar to the provided example. You can do just a rewording. However, remember, that the resulted text must {language}. 
                
                Also, uou must write only the text without any additional comments from yourself. 

            '''
        
        comment = f'''

        The text example is below: 
        """
        {text}
        """

        '''


        prompt_outputs[strat_label].append(analyze_sentiment(comment, system_prompt=system_prompt_overall))



  0%|          | 0/11 [00:00<?, ?it/s]
%|          | 0/858 [00:00<?, ?it/s]
%|          | 1/858 [00:00<13:05,  1.09it/s]
%|          | 2/858 [00:01<10:53,  1.31it/s]
%|          | 3/858 [00:02<09:35,  1.48it/s]
%|          | 4/858 [00:02<09:30,  1.50it/s]
%|          | 5/858 [00:03<08:36,  1.65it/s]
%|          | 6/858 [00:03<08:54,  1.60it/s]
%|          | 7/858 [00:05<10:52,  1.30it/s]
%|          | 8/858 [00:06<12:54,  1.10it/s]
%|          | 9/858 [00:13<38:55,  2.75s/it]
%|          | 10/858 [00:13<29:53,  2.12s/it]
%|▏         | 11/858 [00:16<31:39,  2.24s/it]
%|▏         | 12/858 [00:18<29:50,  2.12s/it]
%|▏         | 13/858 [00:18<23:07,  1.64s/it]
%|▏         | 14/858 [00:19<20:36,  1.47s/it]
%|▏         | 15/858 [00:20<19:22,  1.38s/it]
%|▏         | 16/858 [00:21<16:07,  1.15s/it]
%|▏         | 17/858 [00:22<15:30,  1.11s/it]
%|▏         | 18/858 [00:23<13:05,  1.07it/s]
%|▏         | 19/858 [00:24<13:36,  1.03it/s]
%|▏         | 20/858 [00:24<11:54,  1.17it/s]
%|▏         |

In [41]:
prompt_outputs

{'negative_ua': ['Мені так сумно, коли доводиться чекати в черзі на поштовому відділенні, де обслуговують впівтришки.',
  'Ужасно! Це жахливо... Гірше ніж я собі уявляв.',
  'Замовлення піца було розчаруванням',
  'Зранку прокинулась, а за вікном лише хмари й дощ.',
  'Гидко виглядає',
  'Маруся вчора підвела мене',
  'Моє замовлення виявилося неправильним і небажаним, обслуговування залишає бажати кращого, це реально вразило і засмутило.',
  'Таке враження, нічим немає, лише порожнеча та розчарування. Навіщо це все, коли все одно нічого немає з цього. Жахливо, як можна так обдурити та обманути, це ж абсурд.',
  '❗️Жахлива ситуація в Одесі 15 липня. Рапорт про приголомшливу інформацію  ✅Малинський район: загоряння в кількох закладах харчування. Пожежникам вдалося порятувати лише 10 осіб, евакуювали 40 людей та 57 осіб зникли безвісти. Наразі зафіксовано 15 постраждалих та 3 загиблих, серед яких є дитина, яка померла без свідомості. Пожежу поки неможливо приборкати, тривають вогнеборці.

In [49]:
print(123)

123


In [50]:
print(123132132)

123132132


In [51]:
prompt_outputs.keys()

dict_keys(['negative_ua', 'positive_ua', 'negative_ru', 'neutral_ru', 'mixed_ua', 'positive_ru', 'negative_mixed', 'neutral_mixed', 'mixed_ru', 'positive_mixed', 'mixed_mixed'])

In [52]:
import pickle

def save_prompt_outputs(prompt_outputs, filename='prompt_outputs.pkl'):
    """
    Save the prompt_outputs dictionary to a pickle file.
    
    Args:
        prompt_outputs (dict): The dictionary containing the outputs to save
        filename (str): Name of the pickle file to save to (default: 'prompt_outputs.pkl')
    
    Returns:
        bool: True if the save was successful, False otherwise
    """
    try:
        with open(filename, 'wb') as file:
            pickle.dump(prompt_outputs, file)
        print(f"Successfully saved prompt_outputs to {filename}")
        return True
    except Exception as e:
        print(f"Error saving prompt_outputs: {e}")
        return False

# Example usage
# save_prompt_outputs(prompt_outputs)
# Or with a custom filename:
# save_prompt_outputs(prompt_outputs, 'my_outputs_backup.pkl')

In [53]:
save_prompt_outputs(prompt_outputs, filename='prompt_outputs.pkl')

Successfully saved prompt_outputs to prompt_outputs.pkl


True

In [54]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels_v1,language_gpt,language_manual,language,stratification_label,df_set
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,Ukrainian,ukrainian,ua,neutral_ua,validation
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,Code-mixed,russian,ru,negative_ru,test
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,negative,Code-mixed,russian,ru,negative_ru,train
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,Ukrainian,russian,ua,neutral_ua,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,Code-mixed,ukrainian,ua,negative_ua,train
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,positive,Ukrainian,ukrainian,ua,mixed_ua,train
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,Ukrainian,ukrainian,ua,negative_ua,train


In [56]:
df_augmented = pd.DataFrame({'stratification_label':[], 
             'document_content': []})

In [57]:
for strat_label, texts in tqdm(prompt_outputs.items()):
    temp_df = pd.DataFrame({'stratification_label':[strat_label for i in range(len(texts))], 
             'document_content': texts})

    df_augmented = pd.concat([df_augmented, temp_df], ignore_index=True)

100%|██████████| 11/11 [00:00<00:00, 971.68it/s]


In [60]:
df_augmented.to_parquet('augmentations.parquet')